In [2]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from IPython.display import display, Markdown, Latex
from sklearn.datasets import make_blobs
#%matplotlib widget
from matplotlib.widgets import Slider
from lab_utils_common import dlc
from lab_utils_softmax import plt_softmax
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

> **Note**: Normally, in this course, the notebooks use the convention of starting counts with 0 and ending with N-1,  $\sum_{i=0}^{N-1}$, while lectures start with 1 and end with N,  $\sum_{i=1}^{N}$. This is because code will typically start iteration with 0 while in lecture, counting 1 to N leads to cleaner, more succinct equations. This notebook has more equations than is typical for a lab and thus  will break with the convention and will count 1 to N.

## Softmax Function
In both softmax regression and neural networks with Softmax outputs, N outputs are generated and one output is selected as the predicted category. In both cases a vector $\mathbf{z}$ is generated by a linear function which is applied to a softmax function. The softmax function converts $\mathbf{z}$  into a probability distribution as described below. After applying softmax, each output will be between 0 and 1 and the outputs will add to 1, so that they can be interpreted as probabilities. The larger inputs  will correspond to larger output probabilities.

The softmax function can be written:
$$a_j = \frac{e^{z_j}}{ \sum_{k=1}^{N}{e^{z_k} }} \tag{1}$$
The output $\mathbf{a}$ is a vector of length N, so for softmax regression, you could also write:
\begin{align}
\mathbf{a}(x) =
\begin{bmatrix}
P(y = 1 | \mathbf{x}; \mathbf{w},b) \\
\vdots \\
P(y = N | \mathbf{x}; \mathbf{w},b)
\end{bmatrix}
=
\frac{1}{ \sum_{k=1}^{N}{e^{z_k} }}
\begin{bmatrix}
e^{z_1} \\
\vdots \\
e^{z_{N}} \\
\end{bmatrix} \tag{2}
\end{align}


In [5]:
def my_softmax(z):
    ez=np.exp(z)
    sm=ez/np.sum(ez)
    return (sm)

In [6]:
my_softmax([1,2,3])

array([0.09, 0.24, 0.67])

## Tensorflow
This lab will discuss two ways of implementing the softmax, cross-entropy loss in Tensorflow, the 'obvious' method and the 'preferred' method. The former is the most straightforward while the latter is more numerically stable.

Let's start by creating a dataset to train a multiclass classification model.

In [14]:
# make  dataset for example
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
X_train, y_train = make_blobs(n_samples=2000, centers=centers, cluster_std=1.0,random_state=30)

## The Obvious
The model below is implemented with the softmax as an activation in the final Dense layer.
The loss function is separately specified in the `compile` directive. 

The loss function `SparseCategoricalCrossentropy`. The loss described in (3) above. In this model, the softmax takes place in the last layer. The loss function takes in the softmax output which is a vector of probabilities. 

In [15]:
model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'softmax')    # < softmax activation here
    ]
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    X_train,y_train,
    epochs=10
)
      

Epoch 1/10


2022-07-31 14:03:43.734688: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 2s 4ms/step - loss: 0.9006
Epoch 2/10
63/63 [==============================] - 0s 3ms/step - loss: 0.4347
Epoch 3/10
63/63 [==============================] - 0s 3ms/step - loss: 0.1883
Epoch 4/10
63/63 [==============================] - 0s 4ms/step - loss: 0.1007
Epoch 5/10
63/63 [==============================] - 0s 3ms/step - loss: 0.0686
Epoch 6/10
63/63 [==============================] - 0s 3ms/step - loss: 0.0544
Epoch 7/10
63/63 [==============================] - 0s 4ms/step - loss: 0.0470
Epoch 8/10
63/63 [==============================] - 0s 4ms/step - loss: 0.0415
Epoch 9/10
63/63 [==============================] - 0s 3ms/step - loss: 0.0375
Epoch 10/10
63/63 [==============================] - 0s 3ms/step - loss: 0.0345


Because the softmax is integrated into the output layer, the output is a vector of probabilities.

In [16]:
p_nonpreferred = model.predict(X_train)

63/63 [==============================] - 0s 1ms/step


2022-07-31 14:04:45.127081: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [20]:
print(p_nonpreferred [:2])

[[7.78e-03 6.42e-03 9.71e-01 1.43e-02]
 [9.97e-01 2.57e-03 1.37e-05 1.38e-06]]


In [21]:
print("largest value", np.max(p_nonpreferred), "smallest value", np.min(p_nonpreferred))

largest value 0.9999999 smallest value 3.5595676e-12


## Preferred
In the preferred organization the final layer has a linear activation. For historical reasons, the outputs in this form are referred to as *logits*. The loss function has an additional argument: `from_logits = True`. This informs the loss function that the softmax operation should be included in the loss calculation. This allows for an optimized implementation.

In [25]:
preferred_model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'linear')   #<-- Note
    ]
)
preferred_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #<-- Note
    optimizer=tf.keras.optimizers.Adam(0.001),
)

preferred_model.fit(
    X_train,y_train,
    epochs=10
)
        

Epoch 1/10
28/63 [============>.................] - ETA: 0s - loss: 1.0968

2022-07-31 21:55:18.933964: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 0s 4ms/step - loss: 0.9035
Epoch 2/10
63/63 [==============================] - 0s 4ms/step - loss: 0.4363
Epoch 3/10
63/63 [==============================] - 0s 4ms/step - loss: 0.1888
Epoch 4/10
63/63 [==============================] - 0s 3ms/step - loss: 0.1021
Epoch 5/10
63/63 [==============================] - 0s 4ms/step - loss: 0.0700
Epoch 6/10
63/63 [==============================] - 0s 4ms/step - loss: 0.0549
Epoch 7/10
63/63 [==============================] - 0s 4ms/step - loss: 0.0465
Epoch 8/10
63/63 [==============================] - 0s 4ms/step - loss: 0.0414
Epoch 9/10
63/63 [==============================] - 0s 4ms/step - loss: 0.0376
Epoch 10/10
63/63 [==============================] - 0s 3ms/step - loss: 0.0348


In [26]:
p_preferred = preferred_model.predict(X_train)
print(f"two example output vectors:\n {p_preferred[:2]}")
print("largest value", np.max(p_preferred), "smallest value", np.min(p_preferred))

63/63 [==============================] - 0s 2ms/step
two example output vectors:
 [[-0.61 -1.13  4.19 -0.19]
 [ 8.03  2.17 -3.08 -5.06]]
largest value 14.787938 smallest value -8.373114


2022-07-31 21:56:09.899685: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


## Output:
Note that the output are not probabilities.If the desired output are probabilities, the output should be be processed by a softmax.

In [29]:
sm_preferred = tf.nn.softmax(p_preferred).numpy()
print(f'two example output:\n{sm_preferred[:2]}')
print("largest value", np.max(sm_preferred), "smallest value", np.min(sm_preferred))

two example output:
[[8.02e-03 4.78e-03 9.75e-01 1.22e-02]
 [9.97e-01 2.84e-03 1.49e-05 2.05e-06]]
largest value 0.9999995 smallest value 8.735362e-11


To select the most likely category, the softmax is not required. One can find the index of the largest output using np.argmax().

In [30]:
for i in range(5):
    print(f'{p_preferred[i]},category:{np.argmax(p_preferred[i])}')

[-0.61 -1.13  4.19 -0.19],category:2
[ 8.03  2.17 -3.08 -5.06],category:0
[ 5.7   2.32 -2.24 -4.28],category:0
[-1.46  4.5  -0.23 -3.08],category:1
[ 1.74 -0.61  7.12 -3.77],category:2


## Numerical Stability (optional)
This section discusses some of the methods employed to improve numerical stability. This is for the interested reader and is not at all required.
### Softmax Numerical Stability
The input's  to the softmax are the outputs of a linear layer $z_j = \mathbf{w_j} \cdot \mathbf{x}^{(i)}+b$. These may
be large numbers. The first step of the softmax algorithm computes $e^{z_j}$. This can result in an overflow error if the number gets too large. Try running the cell below:

In [31]:
for z in [500,600,700,800]:
    ez = np.exp(z)
    zs = "{" + f"{z}" + "}"
    print(f"e^{zs} = {ez:0.2e}")

e^{500} = 1.40e+217
e^{600} = 3.77e+260
e^{700} = 1.01e+304
e^{800} = inf


/var/folders/v0/mbv34sgd2xdgkp1dv6283r180000gn/T/ipykernel_22893/1141864107.py:2: RuntimeWarning: overflow encountered in exp
  ez = np.exp(z)


The operation will generate an overflow if the exponent gets too large. Naturally, my_softmax() will generate the same errors:

In [32]:
z_tmp = np.array([[500,600,700,800]])
my_softmax(z_tmp)

/var/folders/v0/mbv34sgd2xdgkp1dv6283r180000gn/T/ipykernel_22893/2441804751.py:2: RuntimeWarning: overflow encountered in exp
  ez=np.exp(z)
/var/folders/v0/mbv34sgd2xdgkp1dv6283r180000gn/T/ipykernel_22893/2441804751.py:3: RuntimeWarning: invalid value encountered in true_divide
  sm=ez/np.sum(ez)


array([[ 0.,  0.,  0., nan]])

Numerical stability can be improved by reducing the size of the exponent. 
Recall 
$$ e^{a + b} = e^ae^b$$
if the $b$  were the opposite sign of $a$, this would reduce the size of the exponent. Specifically, if you multiplied the softmax by a fraction:
$$a_j = \frac{e^{z_j}}{ \sum_{i=1}^{N}{e^{z_i} }} \frac{e^{-b}}{ {e^{-b}}}$$
the exponent would be reduced and the value of the softmax would not change. If $b$ in $e^b$ were the largest value of the $z_j$'s, $max_j(\mathbf{z})$, the exponent would be reduced to its smallest value.
$$\begin{align}
a_j &= \frac{e^{z_j}}{ \sum_{i=1}^{N}{e^{z_i} }} \frac{e^{-max_j(\mathbf{z})}}{ {e^{-max_j(\mathbf{z})}}} \\
&= \frac{e^{z_j-max_j(\mathbf{z})}}{ \sum_{i=1}^{N}{e^{z_i-max_j(\mathbf{z})} }} 
\end{align}$$
It is customary to say $C=max_j(\mathbf{z})$ since the equation would be correct with any constant C. 

$$
a_j = \frac{e^{z_j-C}}{ \sum_{i=1}^{N}{e^{z_i-C} }} \quad\quad\text{where}\quad C=max_j(\mathbf{z})\tag{5}
$$

If we look at our troublesome example where $\mathbf{z}$ contains 500,600,700,800, $C=max_j(\mathbf{z})=800$:
\begin{align}
\mathbf{a}(x) =
\frac{1}{ e^{500-800} + e^{600-800} + e^{700-800} + e^{800-800}}
\begin{bmatrix}
e^{500-800} \\
e^{600-800} \\
e^{700-800} \\
e^{800-800} \\
\end{bmatrix}
= 
\begin{bmatrix}
5.15e-131 \\
1.38e-87 \\
3.7e-44 \\
1.0 \\
\end{bmatrix}
\end{align}

In [33]:
def my_softmax_ns(z):
    """numerically stablility improved"""
    bigz = np.max(z)
    ez = np.exp(z-bigz)              # minimize exponent
    sm = ez/np.sum(ez)
    return(sm)

In [34]:
z_tmp = np.array([500.,600,700,800])
print(tf.nn.softmax(z_tmp).numpy(), "\n", my_softmax_ns(z_tmp))

[5.15e-131 1.38e-087 3.72e-044 1.00e+000] 
 [5.15e-131 1.38e-087 3.72e-044 1.00e+000]


Large values no longer cause an overflow.

### Cross Entropy Loss Numerical Stability
The loss function associated with Softmax, the cross-entropy loss, is repeated here:
\begin{equation}
  L(\mathbf{a},y)=\begin{cases}
    -log(a_1), & \text{if $y=1$}.\\
        &\vdots\\
     -log(a_N), & \text{if $y=N$}
  \end{cases} 
\end{equation}

Where y is the target category for this example and $\mathbf{a}$ is the output of a softmax function. In particular, the values in $\mathbf{a}$ are probabilities that sum to one.
Let's consider a case where the target is two ($y=2$) and just look at the loss for that case. This will result in the loss being:  
$$L(\mathbf{a})= -log(a_2)$$
Recall that $a_2$ is the output of the softmax function described above, so this can be written:
$$L(\mathbf{z})= -log\left(\frac{e^{z_2}}{ \sum_{i=1}^{N}{e^{z_i} }}\right) \tag{6}$$
This can be optimized. However, to make those optimizations, the softmax and the loss must be calculated together as shown in the 'preferred' Tensorflow implementation you saw above.

Starting from (6) above, the loss for the case of y=2:
$log(\frac{a}{b}) = log(a) - log(b)$, so (6) can be rewritten:
$$L(\mathbf{z})= -\left[log(e^{z_2}) - log \sum_{i=1}^{N}{e^{z_i} }\right] \tag{7}$$
The first term can be simplified to just $z_2$:
$$L(\mathbf{z})= -\left[z_2 - log( \sum_{i=1}^{N}{e^{z_i} })\right] =  \underbrace{log \sum_{i=1}^{N}{e^{z_i} }}_\text{logsumexp()} -z_2 \tag{8}$$
It turns out that the $log \sum_{i=1}^{N}{e^{z_i} }$ term in the above equation is so often used, many libraries have an implementation. In Tensorflow this is tf.math.reduce_logsumexp(). An issue with this sum is that the exponent in the sum could overflow if $z_i$ is large. To fix this, we might like to subtract $e^{max_j(\mathbf{z})}$ as we did above, but this will require a bit of work:
$$
\begin{align}
   log \sum_{i=1}^{N}{e^{z_i} } &= log \sum_{i=1}^{N}{e^{(z_i - max_j(\mathbf{z}) + max_j(\mathbf{z}))}} \tag{9}\\
                          &= log \sum_{i=1}^{N}{e^{(z_i - max_j(\mathbf{z}))} e^{max_j(\mathbf{z})}} \\
                          &= log(e^{max_j(\mathbf{z})}) + log \sum_{i=1}^{N}{e^{(z_i - max_j(\mathbf{z}))}} \\
                          &= max_j(\mathbf{z})  + log \sum_{i=1}^{N}{e^{(z_i - max_j(\mathbf{z}))}}
\end{align}
$$
Now, the exponential is less likely to overflow. It is customary to say $C=max_j(\mathbf{z})$ since the equation would be correct with any constant C. We can now write the loss equation:
    
$$L(\mathbf{z})= C+ log( \sum_{i=1}^{N}{e^{z_i-C} }) -z_2  \;\;\;\text{where } C=max_j(\mathbf{z}) \tag{10} $$
A computationally simpler, more stable version of the loss. The above is for an example where the target, y=2 but generalizes to any target.